In [ ]:
# Importing modules
import gurobipy as grb
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from collections import Counter
from itertools import combinations
import datetime

# Function for saving excel files
def save_xls(list_dfs, xls_path, sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

%matplotlib inline

In [ ]:
# Defining class for data retrieval and cleaning
# Import data
enrolls = pd.read_excel('Marshall_Course_Enrollment_1516_1617.xlsx')
cancels = pd.read_excel('Cancelled_Courses_1516_1617.xlsx')
rmcaps = pd.read_excel('Marshall_Room_Capacity_Chart.xlsx')
specprg = pd.read_excel('Summary_Special_Session_Codes_1516_1617.xlsx')
depaloc = pd.read_excel('Department_Allocations_20171.xlsx')
selects = pd.read_excel('Student_Course_Selection_1516.xlsx')


dirname = os.path.dirname(__file__)
filename = os.path.join(dirname, '/relative/path/to/file/you/want')